## 5.5.1 - RDD (Resilient Distributed Dataset)

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/27 14:29:07 WARN Utils: Your hostname, zukui-1984 resolves to a loopback address: 127.0.1.1; using 172.17.212.189 instead (on interface eth0)
24/02/27 14:29:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/27 14:29:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-11 05:05:54|  2020-01-11 05:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.

In [4]:
df_green.rdd

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [5]:
df_green.take(3)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 5, 5, 54), lpep_dropoff_datetime=datetime.datetime(2020, 1, 11, 5, 13, 49), store_and_fwd_flag='N', RatecodeID=1.0, PULocationID=129, DOLocationID=129, passenger_count=1.0, trip_distance=0.81, fare_amount=6.5, extra=0.5, mta_tax=0.5, tip_amount=0.71, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.51, payment_type=1.0, trip_type=1.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 20, 33, 5), lpep_dropoff_datetime=datetime.datetime(2020, 1, 17, 20, 51, 8), store_and_fwd_flag='N', RatecodeID=1.0, PULocationID=75, DOLocationID=42, passenger_count=3.0, trip_distance=2.69, fare_amount=13.5, extra=1.0, mta_tax=0.5, tip_amount=3.06, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=18.36, payment_type=1.0, trip_type=1.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 13, 41), l

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 5, 5, 54), PULocationID=129, total_amount=8.51),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 20, 33, 5), PULocationID=75, total_amount=18.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 13, 41), PULocationID=117, total_amount=43.91),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 22, 25, 44), PULocationID=41, total_amount=12.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 22, 45, 19), PULocationID=129, total_amount=7.8)]

In [9]:
from datetime import datetime

In [13]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [15]:
rdd \
    .filter(filter_outliers) \
    .map(lambda row: row.lpep_pickup_datetime)


PythonRDD[23] at RDD at PythonRDD.scala:53

In [17]:
rows = rdd.take(10)
row = rows[0]

In [18]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 5, 5, 54), PULocationID=129, total_amount=8.51)

In [19]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [20]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 1, 11, 5, 0), 129), (8.51, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 75), (18.36, 1)),
 ((datetime.datetime(2020, 1, 30, 13, 0), 117), (43.91, 1)),
 ((datetime.datetime(2020, 1, 11, 22, 0), 41), (12.3, 1)),
 ((datetime.datetime(2020, 1, 4, 22, 0), 129), (7.8, 1)),
 ((datetime.datetime(2020, 1, 15, 20, 0), 75), (8.76, 1)),
 ((datetime.datetime(2020, 1, 11, 1, 0), 66), (16.26, 1)),
 ((datetime.datetime(2020, 1, 16, 15, 0), 129), (6.8, 1)),
 ((datetime.datetime(2020, 1, 1, 20, 0), 41), (11.8, 1)),
 ((datetime.datetime(2020, 1, 21, 7, 0), 37), (34.04, 1))]

In [ ]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)